# Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from task_three import task_three
from utils import general_utils, data_prep_utils, feature_selection_utils, dimension_reduction_utils

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split


In [2]:
tcga_genes = data_prep_utils.gene_exp_log_trans(data_prep_utils.get_df("tcga_rna", nb=True))
tcga_muts = data_prep_utils.get_df("tcga_mut", nb=True)
tcga_muts_int = data_prep_utils.mut_df_label_to_int(tcga_muts)

# feature selction and dimention redaction

In [ ]:
tcga_genes_filtered, filtered_genes = feature_selection_utils.select_features_by_corrlation(tcga_genes, tcga_muts_int)
tcga_genes_filtered.shape

(167, 3473)

In [3]:
tcga_genes_filtered_norm = data_prep_utils.norm_df(tcga_genes)
vt = VarianceThreshold(0.05)
vt.fit(tcga_genes_filtered_norm)
filtered_genes_df = vt.transform(tcga_genes_filtered_norm)

In [4]:
from sklearn.decomposition import PCA

In [5]:
pca = PCA(n_components=0.95, random_state = 2020) 
pca.fit(filtered_genes_df)
filtered_genes_df_pca = pca.transform(filtered_genes_df)

In [6]:
filtered_genes_df_pca.shape

(167, 125)

# Test build_cor_mat

In [5]:
drugs_dict = {"d1": [0, 1, 2], "d2": [3, 4, 5]}
mut_dict = {"mut1": [1, 2, 3], "mut2": [3, 4, 5], "mut3": [18, 2, 97]}

drugs_df = pd.DataFrame.from_dict(drugs_dict)
print(drugs_df)
mut_df = pd.DataFrame.from_dict(mut_dict)
print(mut_df)

cor_mat = task_three.build_cor_mat(drugs_df, mut_df)
print(cor_mat)
testings_utils.export_drugs_prediction(cor_mat, "test_cor_mat")

   d1  d2
0   0   3
1   1   4
2   2   5
   mut1  mut2  mut3
0     1     3    18
1     2     4     2
2     3     5    97
            d1        d2
mut1  1.000000  1.000000
mut2  1.000000  1.000000
mut3  0.776603  0.776603


# Test get_linear_svc_model

In [3]:
x_train, x_test, y_train, y_test = train_test_split(tcga_genes, tcga_muts, test_size = 0.2, random_state = 42)

In [7]:
svc_model = task_three.get_linear_svc_model(tcga_genes, tcga_muts)

C:\Users\yaelz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


KeyboardInterrupt: 

# Test get_gradient_boosting_class_model

In [26]:
X_train, X_test, y_train, y_test = train_test_split(filtered_genes_df_pca, tcga_muts)

In [28]:
gbc = task_three.get_gradient_boosting_class_model(X_train, y_train)

In [33]:
y_pred = gbc.predict(X_test)

In [35]:
general_utils.get_mse(y_test, y_pred)

0.1471861471861472

#### ROC

In [6]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold

In [7]:
tcga_genes, tcga_muts, beat_rnaseq_shared, beat_drug = task_three.load_df(nb = True)

In [ ]:
mut_predict= task_three.get_mut_predict(tcga_genes, tcga_muts, beat_rnaseq_shared_filterd)

In [ ]:
mut_predict= task_three.get_mut_predict(tcga_genes, tcga_muts, beat_rnaseq_shared_filterd)
print(mut_predict)
mut_drug_df = task_three.build_cor_mat(drugs_filtered, general_utils.convert_predict_to_df(mut_predict, tcga_muts.columns, beat_rnaseq_shared.index))
print(mut_drug_df)